<a href="https://colab.research.google.com/github/jingvf/IDS/blob/main/Pytorch_AttentionGRU_DP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# 加载数据
data_path = '/content/drive/MyDrive/2025paper/dataset/small_dataset.csv'  # 替换为你的数据路径
df = pd.read_csv(data_path)

In [2]:
!pip install opacus -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.4/254.4 kB 12.5 MB/s eta 0:00:00


In [3]:
# 输出类别数量
num_classes = df['Class'].nunique()
print(f"总类别数: {num_classes}")

# 输出每个类别的数量
class_counts = df['Class'].value_counts()
print("每个类别的样本数量：")
print(class_counts)
print(df.head())

总类别数: 5
每个类别的样本数量：
Class
Normal          946844
SpoofingRPM      65490
SpoofingGear     59725
DoS              10553
Fuzzy             8967
Name: count, dtype: int64
      Timestamp CAN ID  DLC Data[0] Data[1] Data[2] Data[3] Data[4] Data[5]  \
0  1.478192e+09    316    8      45      29      24      ff      29      24   
1  1.478192e+09    316    8      45      29      24      ff      29      24   
2  1.478195e+09   0140    8      00      00      00      00      08      28   
3  1.478191e+09   0545    8      d8      00      00      8a      00      00   
4  1.478195e+09   043f    8       1      45      60      ff      6b       0   

  Data[6] Data[7]         Class  
0       0      ff   SpoofingRPM  
1       0      ff   SpoofingRPM  
2      2f      15        Normal  
3      00      00        Normal  
4       0       0  SpoofingGear  


In [4]:
from sklearn.preprocessing import StandardScaler
import torch
from sklearn.preprocessing import LabelEncoder
# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 1. CAN ID 转成 int（十六进制字符串转十进制数字）
df["CAN ID"] = df["CAN ID"].apply(lambda x: int(str(x), 16) if isinstance(x, str) else int(x))

# 2. 提取数值特征 (DLC + Data[0-7]) 并转为十进制
feature_cols = ["DLC"] + [f"Data[{i}]" for i in range(8)]
# Use .map instead of .applymap for consistency, although applymap was the one triggering the warning
X_features = df[feature_cols].astype(str).map(lambda x: int(x, 16) if isinstance(x, str) else int(x)).values

# 3. 标准化
scaler = StandardScaler()
X_features_scaled = scaler.fit_transform(X_features)

# 4. CAN ID 单独提取
X_canid = df["CAN ID"].values
le = LabelEncoder()
y_encoded = le.fit_transform(df["Class"].values)
# Calculate class weights
import numpy as np

counts = [946844, 65490, 59725, 10553, 8967] # Using the counts from the previous output
total = sum(counts)
class_weights = [total/c for c in counts]
class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)

Using device: cuda


In [5]:
from sklearn.model_selection import train_test_split

# ===== 第一步：划分训练集 + 临时集（验证+测试） =====
X_train_cid, X_temp_cid, X_train_feat, X_temp_feat, y_train, y_temp = train_test_split(
    X_canid, X_features_scaled, y_encoded, test_size=0.3, random_state=42
)
# 这里 test_size=0.3 表示 30% 留作验证+测试

# ===== 第二步：从临时集划分验证集和测试集 =====
X_val_cid, X_test_cid, X_val_feat, X_test_feat, y_val, y_test = train_test_split(
    X_temp_cid, X_temp_feat, y_temp, test_size=0.5, random_state=42
)
# test_size=0.5 表示临时集一半做测试，一半做验证

# ===== 输出检查 =====
print("原始数据：")
print(f"X_canid shape = {X_canid.shape}")
print(f"X_features_scaled shape = {X_features_scaled.shape}")
print(f"y_encoded shape = {y_encoded.shape}")

print("\n训练集：")
print(f"CAN IDs shape = {X_train_cid.shape}")
print(f"Features shape = {X_train_feat.shape}")
print(f"Labels shape = {y_train.shape}")

print("\n验证集：")
print(f"CAN IDs shape = {X_val_cid.shape}")
print(f"Features shape = {X_val_feat.shape}")
print(f"Labels shape = {y_val.shape}")

print("\n测试集：")
print(f"CAN IDs shape = {X_test_cid.shape}")
print(f"Features shape = {X_test_feat.shape}")
print(f"Labels shape = {y_test.shape}")

print("\n类别映射：")
print(dict(zip(le.classes_, le.transform(le.classes_))))


原始数据：
X_canid shape = (1091579,)
X_features_scaled shape = (1091579, 9)
y_encoded shape = (1091579,)

训练集：
CAN IDs shape = (764105,)
Features shape = (764105, 9)
Labels shape = (764105,)

验证集：
CAN IDs shape = (163737,)
Features shape = (163737, 9)
Labels shape = (163737,)

测试集：
CAN IDs shape = (163737,)
Features shape = (163737, 9)
Labels shape = (163737,)

类别映射：
{'DoS': np.int64(0), 'Fuzzy': np.int64(1), 'Normal': np.int64(2), 'SpoofingGear': np.int64(3), 'SpoofingRPM': np.int64(4)}


In [6]:
!pip install opacus==1.1.3 -q # 指定稳定版本
import opacus
print(opacus.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.7/181.7 kB 13.7 MB/s eta 0:00:00
1.1.3


In [7]:
import torch
from torch.utils.data import Dataset, DataLoader

# ======================
# 1. 数据集定义
# ======================
SEQ_LEN = 5

class CANSequenceDataset(Dataset):
    def __init__(self, canid, features, labels, seq_len=10):
        self.seq_len = seq_len
        self.canid = torch.tensor(canid, dtype=torch.long)
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels) - self.seq_len + 1

    def __getitem__(self, idx):
        seq_canid = self.canid[idx:idx+self.seq_len].clone().detach()
        seq_features = self.features[idx:idx+self.seq_len].clone().detach()
        label = self.labels[idx+self.seq_len-1].clone().detach()
        return seq_canid, seq_features, label

In [8]:
BATCH_SIZE = 2048*4

train_dataset = CANSequenceDataset(X_train_cid, X_train_feat, y_train, seq_len=SEQ_LEN)
val_dataset   = CANSequenceDataset(X_val_cid, X_val_feat, y_val, seq_len=SEQ_LEN)
test_dataset  = CANSequenceDataset(X_test_cid, X_test_feat, y_test, seq_len=SEQ_LEN)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=False)
test_loader  = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=False)


# 打印一个 batch 的 shape
for batch in train_loader:
    cids, feats, labels = batch
    print("Batch CAN ID shape:", cids.shape)       # (BATCH_SIZE, SEQ_LEN)
    print("Batch Features shape:", feats.shape)    # (BATCH_SIZE, SEQ_LEN, 9)
    print("Batch Labels shape:", labels.shape)     # (BATCH_SIZE,)
    break

Batch CAN ID shape: torch.Size([8192, 5])
Batch Features shape: torch.Size([8192, 5, 9])
Batch Labels shape: torch.Size([8192])


In [13]:
import torch
import torch.nn as nn
from opacus import PrivacyEngine
from opacus.validators import ModuleValidator
from opacus.layers import DPGRU  # 导入Opacus兼容的GRU层

class CNN_DPGRU_Model(nn.Module):
    def __init__(self, num_features, hidden_dim, num_classes,
                 cnn_channels=64, kernel_size=3, num_layers=1, dropout=0.3):
        super(CNN_DPGRU_Model, self).__init__()

        # CNN部分 (仍然是普通的Conv1d，梯度会由PrivacyEngine控制)
        self.cnn = nn.Sequential(
            nn.Conv1d(in_channels=num_features, out_channels=cnn_channels,
                     kernel_size=kernel_size, padding=1),
            nn.ReLU(),
            # Replace BatchNorm1d with GroupNorm
            nn.GroupNorm(num_groups=8, num_channels=cnn_channels), # Using GroupNorm with 8 groups
            nn.Dropout(0.2),
            nn.Conv1d(in_channels=cnn_channels, out_channels=cnn_channels,
                     kernel_size=kernel_size, padding=1),
            nn.ReLU(),
            # Replace BatchNorm1d with GroupNorm
            nn.GroupNorm(num_groups=8, num_channels=cnn_channels), # Using GroupNorm with 8 groups
            nn.Dropout(0.2)
        )

        # DPGRU部分
        self.gru = DPGRU(
            input_size=cnn_channels,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True  # 双向
        )

        # 分类层
        self.fc = nn.Linear(hidden_dim * 2, num_classes)  # 双向 → hidden_dim*2
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # x: (batch_size, seq_len, num_features)
        x = x.transpose(1, 2)  # (batch, features, seq_len)

        # CNN
        cnn_out = self.cnn(x)  # (batch, cnn_channels, seq_len)

        # 送入DPGRU: (batch, seq_len, cnn_channels)
        gru_input = cnn_out.transpose(1, 2)
        gru_out, _ = self.gru(gru_input)

        # 取最后一个时间步
        last_output = gru_out[:, -1, :]  # (batch, hidden_dim*2)

        # 分类
        output = self.fc(self.dropout(last_output))
        return output

In [14]:
import torch
import torch.nn as nn
from opacus.layers import DPGRU, DPMultiheadAttention

class SimpleSelfAttention(nn.Module):
    def __init__(self, hidden_dim, num_heads=4):
        super(SimpleSelfAttention, self).__init__()
        assert hidden_dim % num_heads == 0, "hidden_dim 必须能整除 num_heads"
        self.num_heads = num_heads
        self.head_dim = hidden_dim // num_heads

        # Q, K, V 映射（Linear 支持 DP）
        self.q_proj = nn.Linear(hidden_dim, hidden_dim)
        self.k_proj = nn.Linear(hidden_dim, hidden_dim)
        self.v_proj = nn.Linear(hidden_dim, hidden_dim)

        self.out_proj = nn.Linear(hidden_dim, hidden_dim)

    def forward(self, x):
        # x: (batch, seq_len, hidden_dim)
        B, T, H = x.size()

        # 线性映射
        Q = self.q_proj(x)  # (B, T, H)
        K = self.k_proj(x)
        V = self.v_proj(x)

        # 分头
        Q = Q.view(B, T, self.num_heads, self.head_dim).transpose(1, 2)  # (B, heads, T, head_dim)
        K = K.view(B, T, self.num_heads, self.head_dim).transpose(1, 2)
        V = V.view(B, T, self.num_heads, self.head_dim).transpose(1, 2)

        # Attention score
        scores = (Q @ K.transpose(-2, -1)) / (self.head_dim ** 0.5)  # (B, heads, T, T)
        attn = torch.softmax(scores, dim=-1)

        # 聚合
        out = attn @ V  # (B, heads, T, head_dim)
        out = out.transpose(1, 2).contiguous().view(B, T, H)  # (B, T, H)

        # 输出
        return self.out_proj(out)

class MultiAttn_GRU_Model(nn.Module):
    def __init__(self, num_features, hidden_dim, num_classes,
                 attn_heads=4, num_layers=1, dropout=0.3):
        super(MultiAttn_GRU_Model, self).__init__()

        # DP-GRU (单向)
        self.gru = DPGRU(
            input_size=num_features,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,       # DPGRU支持 batch_first
            bidirectional=False
        )
        self.attn = SimpleSelfAttention(hidden_dim, attn_heads)

        # 分类层
        self.fc = nn.Linear(hidden_dim, num_classes)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # 输入: (batch, seq_len, num_features)
        # 先过GRU
        gru_out, _ = self.gru(x)   # (batch, seq_len, hidden_dim)

        # 注意力 (batch, seq_len, hidden_dim)
        attn_out = self.attn(gru_out)

        # 取最后时间步
        last_output = attn_out[:, -1, :]

        return self.fc(self.dropout(last_output))


In [15]:
# Define the device
print(f"Using device: {device}")
num_features = X_train_feat.shape[1]   # DLC+Data[0-7] → 9
hidden_dim = 128
num_classes = len(set(y_train))

model = MultiAttn_GRU_Model(num_features, hidden_dim, num_classes)
model.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

Using device: cuda


In [16]:
from opacus import PrivacyEngine

EPOCHS = 2
privacy_engine = PrivacyEngine()
model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimizer,
    data_loader=train_loader,
    target_epsilon=5.0,
    target_delta=1e-5,
    epochs=EPOCHS,
    max_grad_norm=1.0
)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from opacus import PrivacyEngine
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt

# ==== 定义模型 ====
class MultiAttGRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_heads, num_classes):
        super().__init__()
        self.gru = nn.GRU(input_dim, hidden_dim, batch_first=True)
        self.attn = nn.MultiheadAttention(hidden_dim, num_heads, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        out, _ = self.gru(x)  # [B, T, H]
        attn_out, _ = self.attn(out, out, out)  # Self-Attention
        pooled = attn_out.mean(dim=1)  # 平均池化
        return self.fc(pooled)

# ==== 训练函数 ====
def train_and_eval(train_loader, test_loader, hidden_dim, num_heads, noise_multiplier, device="cuda"):
    model = MultiAttGRU(input_dim=1, hidden_dim=hidden_dim, num_heads=num_heads, num_classes=2).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.CrossEntropyLoss()

    privacy_engine = PrivacyEngine()
    model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
        module=model,
        optimizer=optimizer,
        data_loader=train_loader,
        epochs=3,
        target_epsilon=5,
        target_delta=1e-5,
        max_grad_norm=1.0,
    )

    for epoch in range(3):  # 小规模实验即可
        model.train()
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            pred = model(x)
            loss = criterion(pred, y)
            loss.backward()
            optimizer.step()

    # ==== 测试 ====
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            y_true.extend(y.cpu().numpy())
            y_pred.extend(pred.argmax(1).cpu().numpy())
    return accuracy_score(y_true, y_pred), f1_score(y_true, y_pred)

# ==== 超参数网格搜索 ====
search_space = {
    "hidden_dim": [64, 128, 256],
    "num_heads": [2, 4, 8],
    "noise_multiplier": [0.5, 1.0, 1.5]
}

results = []
for hd in search_space["hidden_dim"]:
    for nh in search_space["num_heads"]:
        for nm in search_space["noise_multiplier"]:
            acc, f1 = train_and_eval(train_loader, test_loader, hd, nh, nm)
            results.append((hd, nh, nm, acc, f1))
            print(f"hidden_dim={hd}, num_heads={nh}, noise={nm} => Acc={acc:.4f}, F1={f1:.4f}")

# ==== 可视化 ====
import pandas as pd
df = pd.DataFrame(results, columns=["hidden_dim", "num_heads", "noise", "Acc", "F1"])
pivot = df.pivot_table(index="hidden_dim", columns="noise", values="Acc")
pivot.plot(kind="bar", title="Accuracy vs Hyperparameters")
plt.ylabel("Accuracy")
plt.show()

print("最优参数：")
print(df.sort_values(by="F1", ascending=False).head(1))


In [17]:
import torch.optim as optim
from tqdm import tqdm


for epoch in range(EPOCHS):
    model.train()
    total_loss, total_correct = 0, 0

    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}", leave=False)
    for cids, feats, labels in progress_bar:
        feats, labels = feats.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(feats)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        preds = torch.argmax(outputs, dim=1)
        total_loss += loss.item() * labels.size(0)
        total_correct += (preds == labels).sum().item()
        progress_bar.set_postfix({'loss': total_loss/((progress_bar.n+1)*BATCH_SIZE), 'acc': total_correct/((progress_bar.n+1)*BATCH_SIZE)})

    train_loss = total_loss / len(train_dataset)
    train_acc  = total_correct / len(train_dataset)
    print(f"Epoch {epoch+1} → Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")


    # -------------------------------
    # 验证集
    # -------------------------------
    model.eval()
    val_correct, val_total = 0, 0
    with torch.no_grad():
        for cids, feats, labels in val_loader:
            feats, labels = feats.to(device), labels.to(device)
            outputs = model(feats)
            preds = torch.argmax(outputs, dim=1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)
    val_acc = val_correct / val_total

    # 打印每个 epoch 结果
    print(f"Epoch {epoch+1} → Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

    # 获取详细的隐私账户信息
    accountant = privacy_engine.accountant
    # 方法1：获取当前epsilon值
    epsilon = accountant.get_epsilon(delta=1e-5)
    print(f"11Current epsilon: {epsilon}")
    # 方法2：获取完整的隐私开销信息
    privacy_spent = accountant.get_privacy_spent(delta=1e-5)
    print("f222Privacy spent: {privacy_spent}")


# -------------------------------
# 测试集最终评估
# -------------------------------
model.eval()
test_correct, test_total = 0, 0
with torch.no_grad():
    for cids, feats, labels in test_loader:
        feats, labels = feats.to(device), labels.to(device)
        outputs = model(feats)
        preds = torch.argmax(outputs, dim=1)
        test_correct += (preds == labels).sum().item()
        test_total += labels.size(0)

test_acc = test_correct / test_total
print(f"\nFinal Test Accuracy: {test_acc:.4f}")

Epoch 1/2:   0%|          | 0/92 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py:1864: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


Epoch 1 → Train Loss: 1.0467, Train Acc: 0.8310
Epoch 1 → Train Loss: 1.0467, Train Acc: 0.8310, Val Acc: 0.8678
11Current epsilon: 4.401611641339559
f222Privacy spent: {privacy_spent}


Epoch 2 → Train Loss: 0.6996, Train Acc: 0.8574
Epoch 2 → Train Loss: 0.6996, Train Acc: 0.8574, Val Acc: 0.8678
11Current epsilon: 5.020429419685078
f222Privacy spent: {privacy_spent}

Final Test Accuracy: 0.8676


After installing the library, please run the code cell again.